In [1]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp

In [2]:
baseline = np.load('images0001.npy')
kaggle = pd.read_parquet('5414471.parquet')


In [3]:
# ordered body_data, face_data, lh_data, rh_data, body_world_data
# body_data
len(baseline)


586

In [4]:
len([s for s in kaggle.columns if "x" in s])
#pose, face, lhand, rhand

543

In [55]:
mp_holistic = mp.solutions.holistic

BG_COLOR = (192, 192, 192) # gray

BODY_POINTS = mp_holistic.PoseLandmark._member_names_
BODY_LIMBS = [(int(a), int(b)) for a, b in mp_holistic.POSE_CONNECTIONS]

HAND_POINTS = mp_holistic.HandLandmark._member_names_
HAND_LIMBS = [(int(a), int(b)) for a, b in mp_holistic.HAND_CONNECTIONS]

FACE_POINTS_NUM = lambda additional_points=0: additional_points + 468
FACE_POINTS = lambda additional_points=0: [str(i) for i in range(FACE_POINTS_NUM(additional_points))]
FACE_LIMBS = [(int(a), int(b)) for a, b in mp_holistic.FACEMESH_TESSELATION]

def component_points(component, width: int, height: int, num: int):
    if component is not None:
        lm = component.landmark
        return np.array([[p.x * width, p.y * height, p.z] for p in lm]), np.ones(num)

    return np.zeros((num, 3)), np.zeros(num)

def body_points(component, width: int, height: int, num: int):
    if component is not None:
        lm = component.landmark
        return np.array([[p.x * width, p.y * height, p.z] for p in lm]), np.array([p.visibility for p in lm])

    return np.zeros((num, 3)), np.zeros(num)

w_refine = []

with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=False,
    refine_face_landmarks=True) as holistic:

    datas = []
    confs = []
    file = "test.jpg"
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    h, w, _ = image.shape
    
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    #body_data, body_confidence = body_points(results.pose_landmarks, w, h, 33)
    face_data, face_confidence = component_points(results.face_landmarks, w, h,
                                            FACE_POINTS_NUM(10))
    #lh_data, lh_confidence = component_points(results.left_hand_landmarks, w, h, 21)
    #rh_data, rh_confidence = component_points(results.right_hand_landmarks, w, h, 21)
    #body_world_data, body_world_confidence = body_points(results.pose_world_landmarks, w, h, 33)
    
    #data = np.concatenate([body_data, face_data, lh_data, rh_data, body_world_data])
    #conf = np.concatenate([body_confidence, face_confidence, lh_confidence, rh_confidence, body_world_confidence])
    
    #datas.append(data)
    #confs.append(conf)
    
    #numpy_res = np.concatenate((data,conf[:,None]),axis= 1)
    #np.save("face.npy",numpy_res)
    w_refine = (face_data)

n_refine = []

with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=False,
    refine_face_landmarks=False) as holistic:

    datas = []
    confs = []
    file = "test.jpg"
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    h, w, _ = image.shape
    
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    #body_data, body_confidence = body_points(results.pose_landmarks, w, h, 33)
    face_data, face_confidence = component_points(results.face_landmarks, w, h,
                                            FACE_POINTS_NUM(10))
    #lh_data, lh_confidence = component_points(results.left_hand_landmarks, w, h, 21)
    #rh_data, rh_confidence = component_points(results.right_hand_landmarks, w, h, 21)
    #body_world_data, body_world_confidence = body_points(results.pose_world_landmarks, w, h, 33)
    
    #data = np.concatenate([body_data, face_data, lh_data, rh_data, body_world_data])
    #conf = np.concatenate([body_confidence, face_confidence, lh_confidence, rh_confidence, body_world_confidence])
    
    #datas.append(data)
    #confs.append(conf)
    
    #numpy_res = np.concatenate((data,conf[:,None]),axis= 1)
    #np.save("face.npy",numpy_res)
    n_refine = (face_data)

I0000 00:00:1713380496.210006  672196 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713380496.221297  718254 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.67), renderer: NVIDIA GeForce GTX 745/PCIe/SSE2
I0000 00:00:1713380496.472284  672196 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713380496.480801  718267 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.67), renderer: NVIDIA GeForce GTX 745/PCIe/SSE2


False

In [64]:
w_refine

array([[ 6.88365479e+02,  2.30633583e+02, -1.63010526e-02],
       [ 6.89290085e+02,  2.02337158e+02, -2.77429167e-02],
       [ 6.90590973e+02,  2.10589135e+02, -1.49651533e-02],
       ...,
       [ 7.30019989e+02,  1.44913294e+02,  2.72272527e-03],
       [ 7.22594604e+02,  1.51988661e+02,  2.72272527e-03],
       [ 7.29972763e+02,  1.58975569e+02,  2.72272527e-03]])

In [63]:
n_refine

array([[ 6.89000168e+02,  2.29480362e+02, -1.60203390e-02],
       [ 6.85245361e+02,  2.02501760e+02, -2.83331759e-02],
       [ 6.88125687e+02,  2.11214540e+02, -1.51533317e-02],
       ...,
       [ 7.01935883e+02,  1.57873954e+02, -3.88932042e-03],
       [ 7.44121704e+02,  1.49670632e+02,  1.35842024e-03],
       [ 7.48168564e+02,  1.46422563e+02,  1.03706680e-03]])

In [8]:
np.all(baseline[33:511][:,:-1]==face_data)

True

In [9]:
np.all(baseline[511:532][:,:-1]==lh_data)

True

In [10]:
np.all(baseline[532:553][:,:-1]==rh_data)

True

In [70]:
import matplotlib.pyplot as plt

# Sample array of points
points = baseline[511:532]

# Extract x and y coordinates from the points
x_values = [point[0] for point in points]
y_values = [point[1] for point in points]

# Plot the points
plt.scatter(x_values, y_values, color='blue')
plt.title('Array of Points')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.grid(True)
plt.show()

NameError: name 'baseline' is not defined

In [11]:
np.all(baseline[553:586][:,:-1]==body_world_data)

True

In [12]:
baseline

array([[ 6.88897781e+02,  1.99019179e+02, -3.72468978e-01,
         9.98877823e-01],
       [ 7.14261780e+02,  1.61515503e+02, -3.41200978e-01,
         9.97768521e-01],
       [ 7.33020248e+02,  1.61795697e+02, -3.41530502e-01,
         9.97816801e-01],
       ...,
       [-1.26243916e+02,  4.57067599e+02,  3.65407802e-02,
         2.63812626e-03],
       [ 2.29747534e+00,  4.93215108e+02,  6.20303825e-02,
         8.92267330e-04],
       [-1.36530371e+02,  4.69190712e+02, -7.62456236e-03,
         9.71628353e-04]])

In [13]:
kaggle


,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1816796431,0,0.710588,0.699951,0.705657,0.691768,0.699669,0.701980,0.709724,0.610405,0.712660,...,-0.245855,-0.269148,-0.129743,-0.251501,-0.278687,-0.266530,-0.152852,-0.257519,-0.275822,-0.266876
1816796431,1,0.709525,0.697582,0.703713,0.691016,0.697576,0.700467,0.709796,0.616540,0.713729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1816796431,2,0.711059,0.700858,0.706272,0.693285,0.700825,0.703319,0.711549,0.615606,0.715143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1816796431,3,0.712799,0.702518,0.707840,0.694899,0.702445,0.704794,0.712483,0.625044,0.715677,...,-0.370770,-0.408097,-0.185217,-0.325494,-0.343373,-0.328294,-0.203126,-0.315719,-0.326104,-0.314282
1816796431,4,0.712349,0.705451,0.709918,0.696006,0.705180,0.706928,0.712685,0.614356,0.714875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848182207,296,0.657136,0.635888,0.643259,0.619031,0.633084,0.631827,0.630708,0.533120,0.626672,...,-0.143147,-0.139659,-0.066276,-0.130910,-0.127341,-0.106674,-0.083439,-0.124994,-0.119394,-0.101404
1848182207,297,0.655706,0.635570,0.642730,0.618637,0.632830,0.631554,0.630344,0.531868,0.626445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1848182207,298,0.653681,0.636057,0.643054,0.618643,0.633258,0.631800,0.630059,0.531178,0.625990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# x -> y -> z
#face -> lhand -> pose -> rhand
X_COLS=[f'x_face_{i}' for i in range(478)]+[f'x_left_hand_{i}' for i in range(21)]+[f'x_pose_{i}' for i in range(33)]+[f'x_right_hand_{i}' for i in range(21)]
Y_COLS=[f'y_face_{i}' for i in range(478)]+[f'y_left_hand_{i}' for i in range(21)]+[f'y_pose_{i}' for i in range(33)]+[f'y_right_hand_{i}' for i in range(21)]
Z_COLS=[f'z_face_{i}' for i in range(478)]+[f'z_left_hand_{i}' for i in range(21)]+[f'z_pose_{i}' for i in range(33)]+[f'z_right_hand_{i}' for i in range(21)]

#also need sequence ID
ALL_COLS=X_COLS + Y_COLS + Z_COLS
len(ALL_COLS)

ALL_COLS = ["phrase", "frame"] + ALL_COLS
#difference of 30 from the 10 extra face points

In [62]:
#this will be the final thing

df = pd.DataFrame(columns=ALL_COLS)

In [96]:
%%time

data = pd.read_csv("final.csv")
import warnings
warnings.filterwarnings('ignore')

#iterates starting from a Batch folder
def combine_numpy_files(folder_path):
    df = pd.DataFrame(columns=ALL_COLS)
    # List all directories in the given folder
    directories = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    failed = []
    # Iterate through each directory
    early_break = 0
    for directory in directories:
        if early_break >  25:
            break
        dir_path = os.path.join(folder_path, directory)
        target = (folder_path + '/' + directory)[8:]
        print(target)

        numpy_files = [f for f in os.listdir(dir_path) if f.endswith('.npy')]

        try:
            phrase = data[data['filename']==target]['label_proc'].iloc[0]
            scale_width = data[data['filename']==target]['width'].iloc[0]
            scale_height = data[data['filename']==target]['height'].iloc[0]
        except:
            failed.append(target)
            continue
            
        arrays = [np.load(os.path.join(dir_path, file)) for file in numpy_files]
        
        frame = 0
        for arr in arrays:
            arr = arr[:-33]
            #rearranging time
            x = arr[:,0]
            y = arr[:,1]
            z = arr[:,2]
            x_rearr = np.concatenate((x[33:511] , x[511:532] , x[0:33] , x[532:553])) / scale_width
            y_rearr = np.concatenate((y[33:511] , y[511:532] , y[0:33] , y[532:553])) / scale_height
            z_rearr = np.concatenate((z[33:511] , z[511:532] , z[0:33] , z[532:553]))
            
            fin_row = np.concatenate((x_rearr, y_rearr, z_rearr))
            
            #process the array into a dict for pd concatenation
            ihateeverything = {key: value for key, value in zip(ALL_COLS, fin_row)}
            
            ihateeverything['phrase'] = phrase
            ihateeverything['frame'] = frame

            # add to the df
            new = pd.DataFrame([ihateeverything])
            df = pd.concat([df, new], ignore_index=True)

            frame += 1
        early_break += 1
    
    print(failed)
    return df
        
batch_directory = 'dataset/Batch10'
df = combine_numpy_files(batch_directory)





Batch10/Annual_Report_Saskatchew_2_A5MYYH3B448OY_2
Batch10/IADs_Priorities_2017-2019_2_AIU1NJREW2JSG_0
Batch10/SNASL-Crustaceans_0_AIU1NJREW2JSG_7
Batch10/Review_of_The_Walking_27_AIU1NJREW2JSG_0
Batch10/IADs_Response_to_John_Millers_5_A5MYYH3B448OY_8
Batch10/Review_of_The_Walking_13_A5MYYH3B448OY_0
Batch10/PART_TWO-Interview_Lewis_6_A5MYYH3B448OY_0
Batch10/Annual_Report_Saskatchew_2_A5MYYH3B448OY_3
Batch10/Interview_with_Teraca_Florence_6_AIU1NJREW2JSG_6
Batch10/IADs_Vice_President_Kevin_Ryan_1_AIU1NJREW2JSG_1
Batch10/SNASL-Butterflies_barometers_5_A1YHAWNKMBBOXM_11
Batch10/Review_of_The_Walking_3_A5MYYH3B448OY_1
Batch10/IAD_Believes_in_Transparency_2_AIU1NJREW2JSG_1
Batch10/Annual_Report_Saskatchew_15_A5MYYH3B448OY_0
Batch10/IAD_Submits_Letter_to_Governor_0_A5MYYH3B448OY_5
Batch10/Interview_with_Brian_Lucas_1_A1YHAWNKMBBOXM_11
Batch10/Review_of_The_Walking_0_A5MYYH3B448OY_2
Batch10/Interview_with_Brian_Lucas_1_A18LZUX7ES0QI9_6
Batch10/Interview_with_Teraca_Florence_13_AIU1NJREW2JSG_2

In [94]:
 df

,phrase,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,nev,0,0.687509,0.681790,0.684252,0.684705,0.686646,0.658011,0.686764,0.686266,...,-0.009754,-0.026526,-0.030417,-0.026829,-0.013089,-0.025378,-0.025140,-0.020649,NaN,NaN
1,nev,1,0.687301,0.681171,0.683387,0.683910,0.686181,0.658457,0.686337,0.685761,...,-0.009062,-0.023195,-0.023332,-0.018739,-0.013546,-0.022881,-0.020210,-0.015123,NaN,NaN
2,nev,2,0.687213,0.682233,0.684513,0.685142,0.687519,0.658004,0.688106,0.687908,...,-0.010555,-0.027065,-0.031069,-0.027423,-0.013836,-0.025230,-0.024988,-0.020483,NaN,NaN
3,nev,3,0.689132,0.684904,0.687596,0.688108,0.689950,0.659513,0.690644,0.690721,...,-0.020025,-0.030954,-0.028959,-0.026668,-0.027362,-0.032425,-0.029668,-0.027164,NaN,NaN
4,nev,4,0.687310,0.681456,0.683620,0.684202,0.686547,0.658366,0.686751,0.686230,...,-0.009279,-0.023426,-0.023565,-0.018921,-0.013562,-0.022716,-0.019841,-0.014588,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,arts,14,0.392274,0.389148,0.394296,0.394016,0.392739,0.353198,0.392625,0.392875,...,-0.022159,-0.026881,-0.019123,-0.014097,-0.027601,-0.028878,-0.021489,-0.016222,NaN,NaN
508,arts,15,0.374990,0.370925,0.375401,0.375370,0.375297,0.335548,0.375276,0.375293,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
509,arts,16,0.393843,0.391768,0.396828,0.396635,0.395329,0.354364,0.395566,0.396158,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
510,arts,17,0.374943,0.371262,0.375436,0.375566,0.375895,0.336506,0.376117,0.376233,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN


In [95]:
df.to_csv("test.csv", index=False)